# 我们使用这个notebook解释bAbI task和数据集

# 1. bAbI 任务

## Task 1: Single Supporting Fact

**Facts:**
1. Mary went to the bathroom
2. John moved to the hallway
3. **Mary travelled to the office** (supporting fact)

**Q:** Where is Mary?

**A:** office

## Task 2: Two Supporting Fact

**Facts:**
1. **John** is in the **playground** (supporting fact 1)
2. **John** picked up the **football** (supporting fact 2)
3. Bob went to the kitchen

**Q:** Where is the **football**?

**A:** **playground**

## Task 17: Positional Reasoning

1. The **triangle** is to the right of the **blue square**.
2. The **red square** is on top of the **blue square**.
3. The red sphere is to the right of the blue square.


**Q:** Is the red square to the left of the triangle?

**A:** Yes.

![alt text](figure/positional_example1.png)
或者
![alt text](figure/positional_example2.png)

## Task 19: Path Finding

**Facts:**
1. The kitchen is north of the hallway.
2. The batchroom is west of the bedroom
3. The den is east of the hallway.
4. The office is south of the bedroom.

**Q:** How do you go fro mden to kitchen?

**A:** west, north

**Q:** How do you go from office to batchroom? 

**A:** north, west

![alt text](figure/path-example.png)

## Task 20: Agent't Motivations
1. **John** is **hungry**.
2. **John** goes to the **kitchen**.
3. John grabbed the apple there.
4. Daniel is hungry.

**Q:** Where does Danial go? 
    
**A:** kitchen
    
**Q:** Why did John go to the kitchen?
    
**A:** hungry

# bAbI数据

### bAbI数据的每一个样本包含多个“描述信息”或者“提问题”的句子

所有的句子可以分为这样四类，一个句子或者是C（context）类，或者是Q+S+A类，即一行里面包含（问题，答案，提示），提示是一个或几个整数，意思是答案可以在哪一行里面得到：

* **`C`**: context, 例如 
  + 1 **Mary** moved to the bathroom.
  + 4 **Daniel** went back to the hallway.
* **`Q`**: question, 例如
  + 3 Where is **Mary**?
  + 6 Where is **Daniel**?
* **`A`**: answer
  + 3 **bathroom**
  + 6 **hallway**
* **`S`**: supporting sentences
  + 3 **1**
  + 6 **4**


### bAbI 任务1样本（1）
1 **Mary** moved to the bathroom.

2 **John** went to the hallway.

3 Where is **Mary**? **bathroom	 1**

4 **Daniel** went back to the hallway.

5 **Sandra** moved to the garden.

6 Where is **Daniel**? 	**hallway	4**

7 **John** moved to the office.

8 **Sandra** journeyed to the bathroom.

9 Where is **Daniel**? 	**hallway	4**

10 **Mary** moved to the hallway.

11 **Daniel** travelled to the office.

12 Where is **Daniel**? 	**office	11**

13 **John** went back to the garden.

14 **John** moved to the bedroom.

15 Where is **Sandra**? 	**bathroom	8**

### bAbI 任务1样本（2）
1 **Sandra** travelled to the office.

2 **Sandra** went to the bathroom.

3 Where is **Sandra**? 	**bathroom 2**

4 **Mary** went to the bedroom.

5 **Daniel** moved to the hallway.

6 Where is **Sandra**? 	**bathroom 2**

7 **John** went to the garden.

8 **John** travelled to the office.

9 Where is **Sandra**? 	**bathroom 2**

10 **Daniel** journeyed to the bedroom.

11 **Daniel** travelled to the hallway.

12 Where is **John**? 	**office 8**

13 **John** went to the bedroom.

14 **John** travelled to the office.

15 Where is **Daniel**? **hallway 11**



In [3]:
import os as os
import numpy as np

In [4]:
# can be sentence or word
input_mask_mode = "sentence"

In [5]:
def init_babi(fname):
    print("==> Loading test from %s" % fname)
    tasks = []
    task = None
    for i, line in enumerate(open(fname)):
        id = int(line[0:line.find(' ')])
        # 见上面的样本的例子，当ID=1的时候，意味着开始一个下一个样本
        if id == 1:
            task = {"C": "", "Q": "", "A": "", "S": ""}
            # C: context
            # Q: question
            # A: answer
            # S: supporting sentences
            counter = 0
            id_map = {}

        line = line.strip()
        line = line.replace('.', ' . ')
        line = line[line.find(' ')+1:]
        # if not a question
        if line.find('?') == -1:
            task["C"] += line
            id_map[id] = counter
            counter += 1
        else:
            idx = line.find('?')
            tmp = line[idx+1:].split('\t')
            task["Q"] = line[:idx]
            task["A"] = tmp[1].strip()
            task["S"] = []
            for num in tmp[2].split():
                task["S"].append(id_map[int(num.strip())])
            tasks.append(task.copy())

    return tasks

### Demo：使用 `init_babi()` 读入&打印一些样本:

In [6]:
fname1 = '/home/dong/Dropbox/Projects/NLP/memoryNet/bAbIdata/tasks_1-20_v1-2/en/qa1_single-supporting-fact_train.txt'
tasks1 = init_babi(fname1)
print('--examples of task1:')
for i in range(5):
    print('\texample %d' % i)
    print('\t%s' % tasks1[i])

print('\n')
fname2 = '/home/dong/Dropbox/Projects/NLP/memoryNet/bAbIdata/tasks_1-20_v1-2/en/qa2_two-supporting-facts_train.txt'
tasks2 = init_babi(fname2)
print('--example of task2')
for i in range(5):
    print('\texample %d' % i)
    print('\t%s' % tasks2[i])

==> Loading test from /home/dong/Dropbox/Projects/NLP/memoryNet/bAbIdata/tasks_1-20_v1-2/en/qa1_single-supporting-fact_train.txt
--examples of task1:
	example 0
	{'C': 'Mary moved to the bathroom . John went to the hallway . ', 'Q': 'Where is Mary', 'A': 'bathroom', 'S': [0]}
	example 1
	{'C': 'Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden . ', 'Q': 'Where is Daniel', 'A': 'hallway', 'S': [2]}
	example 2
	{'C': 'Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden . John moved to the office . Sandra journeyed to the bathroom . ', 'Q': 'Where is Daniel', 'A': 'hallway', 'S': [2]}
	example 3
	{'C': 'Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden . John moved to the office . Sandra journeyed to the bathroom . Mary moved to the hallway . Daniel travelled to the office . ', 'Q': 'Where is 

## bAbI数据集的20个任务的内容

1) en/ 和en-10k/ 20个任务的数据集，英语，可读
2) {en-valid/， en-valid-10k/} 和 {en/, en10k/}相同的样本，被按照90%，10%的比例分割成training和validation数据集。


| 文件 | 样本数量(en)|样本数量(en-10K)|
| ----------------------------------- | ----------- | ----------- | 
|**qa1_single-supporting-fact**_test.txt | 3000| 3000| 
|**qa1_single-supporting-fact**_train.txt | 3000| 30000|
|qa2_two-supporting-facts_test.txt | 5398|5398|
|qa2_two-supporting-facts_train.txt | 5338|53992|
|qa3_three-supporting-facts_test.txt | 15710|15710|
|qa3_three-supporting-facts_train.txt | 15796|158786|
|qa4_two-arg-relations_test.txt | 3000|3000|
|qa4_two-arg-relations_train.txt | 3000|30000|
|qa5_three-arg-relations_test.txt | 6290|6290|
|qa5_three-arg-relations_train.txt | 6038|64532|
|qa6_yes-no-questions_test.txt | 3050|3050|
|qa6_yes-no-questions_train.txt | 3066|30612|
|qa7_counting_test.txt | 3470|3470|
|qa7_counting_train.txt | 3638|36044|
|qa8_lists-sets_test.txt | 3710|3710|
|qa8_lists-sets_train.txt | 3634|36684|
|qa9_simple-negation_test.txt | 3000|3000|
|qa9_simple-negation_train.txt | 3000|30000|
|qa10_indefinite-knowledge_test.txt | 3000|3000|
|qa10_indefinite-knowledge_train.txt | 3000|30000|
|qa11_basic-coreference_test.txt | 3000|3000|
|qa11_basic-coreference_train.txt | 3000|30000|
|qa12_conjunction_test.txt | 3000|3000|
|qa12_conjunction_train.txt | 3000|30000|
|qa13_compound-coreference_test.txt | 3000|3000|
|qa13_compound-coreference_train.txt | 3000|30000|
|qa14_time-reasoning_test.txt | 3352|3352|
|qa14_time-reasoning_train.txt | 3372|33404|
|qa15_basic-deduction_test.txt | 3000|3000|
|qa15_basic-deduction_train.txt | 3000|30000|
|qa16_basic-induction_test.txt | 10000|10000|
|qa16_basic-induction_train.txt | 10000|100000|
|qa17_positional-reasoning_test.txt | 1250|1250|
|qa17_positional-reasoning_train.txt | 1250|12500|
|qa18_size-reasoning_test.txt | 2210|2210|
|qa18_size-reasoning_train.txt | 2230|22445|
|qa19_path-finding_test.txt | 6000|6000|
|qa19_path-finding_train.txt | 6000|60000|
|qa20_agents-motivations_test.txt | 2000|2000|
|qa20_agents-motivations_train.txt | 2000|20000|

3) shuffled/ -- 词语中的字母随机乱序处理过，不可读（被人理解），模拟学习全新一门语言的情形

 


In [90]:
def init_babi(fname):

    print("==> Loading test from %s" % fname)
    tasks = []
    task = None
    for i, line in enumerate(open(fname)):
        id = int(line[0:line.find(' ')])
        if id == 1:
            task = {"C": "", "Q": "", "A": "", "S": ""}
            counter = 0
            id_map = {}

        line = line.strip()
        line = line.replace('.', ' . ')
        line = line[line.find(' ')+1:]
        # if not a question
        if line.find('?') == -1:
            task["C"] += line
            id_map[id] = counter
            counter += 1

        else:
            idx = line.find('?')
            tmp = line[idx+1:].split('\t')
            task["Q"] = line[:idx]
            task["A"] = tmp[1].strip()
            task["S"] = []
            for num in tmp[2].split():
                task["S"].append(id_map[int(num.strip())])
            tasks.append(task.copy())

    return tasks


def get_babi_raw(id, test_id):
    babi_map = {
        "1": "qa1_single-supporting-fact",
        "2": "qa2_two-supporting-facts",
        "3": "qa3_three-supporting-facts",
        "4": "qa4_two-arg-relations",
        "5": "qa5_three-arg-relations",
        "6": "qa6_yes-no-questions",
        "7": "qa7_counting",
        "8": "qa8_lists-sets",
        "9": "qa9_simple-negation",
        "10": "qa10_indefinite-knowledge",
        "11": "qa11_basic-coreference",
        "12": "qa12_conjunction",
        "13": "qa13_compound-coreference",
        "14": "qa14_time-reasoning",
        "15": "qa15_basic-deduction",
        "16": "qa16_basic-induction",
        "17": "qa17_positional-reasoning",
        "18": "qa18_size-reasoning",
        "19": "qa19_path-finding",
        "20": "qa20_agents-motivations",
        "MCTest": "MCTest",
        "19changed": "19changed",
        "joint": "all_shuffled",
        "sh1": "../shuffled/qa1_single-supporting-fact",
        "sh2": "../shuffled/qa2_two-supporting-facts",
        "sh3": "../shuffled/qa3_three-supporting-facts",
        "sh4": "../shuffled/qa4_two-arg-relations",
        "sh5": "../shuffled/qa5_three-arg-relations",
        "sh6": "../shuffled/qa6_yes-no-questions",
        "sh7": "../shuffled/qa7_counting",
        "sh8": "../shuffled/qa8_lists-sets",
        "sh9": "../shuffled/qa9_simple-negation",
        "sh10": "../shuffled/qa10_indefinite-knowledge",
        "sh11": "../shuffled/qa11_basic-coreference",
        "sh12": "../shuffled/qa12_conjunction",
        "sh13": "../shuffled/qa13_compound-coreference",
        "sh14": "../shuffled/qa14_time-reasoning",
        "sh15": "../shuffled/qa15_basic-deduction",
        "sh16": "../shuffled/qa16_basic-induction",
        "sh17": "../shuffled/qa17_positional-reasoning",
        "sh18": "../shuffled/qa18_size-reasoning",
        "sh19": "../shuffled/qa19_path-finding",
        "sh20": "../shuffled/qa20_agents-motivations",
    }
    if (test_id == ""):
        test_id = id
    babi_name = babi_map[id]
    babi_test_name = babi_map[test_id]
    #babi_train_raw = init_babi(os.path.join(os.path.dirname(os.path.realpath(__file__)), 'data/en-10k/%s_train.txt' % babi_name))
    #babi_test_raw = init_babi(os.path.join(os.path.dirname(os.path.realpath(__file__)), 'data/en-10k/%s_test.txt' % babi_test_name))
    babi_train_raw = init_babi(os.path.join(os.path.realpath('../bAbIdata/tasks_1-20_v1-2/en'), '%s_train.txt' % babi_name))
    babi_test_raw = init_babi(os.path.join(os.path.realpath('../bAbIdata/tasks_1-20_v1-2/en'), '%s_test.txt' % babi_test_name))
    return babi_train_raw, babi_test_raw


def load_glove(dim):
    word2vec = {}

    print("==> loading glove")
    with open(("./data/glove/glove.6B/glove.6B." + str(dim) + "d.txt")) as f:
        for line in f:
            l = line.split()
            word2vec[l[0]] = map(float, l[1:])

    print("==> glove is loaded")

    return word2vec

def create_vector(word, word2vec, word_vector_size, silent=True):
    # if the word is missing from Glove, create some fake vector and store in glove!
    vector = np.random.uniform(0.0,1.0,(word_vector_size,))
    word2vec[word] = vector
    if (not silent):
        print("utils.py::create_vector => %s is missing" % word)
    return vector


def process_word(word, word2vec, vocab, ivocab, 
    word_vector_size, to_return="word2vec", silent=True):
    if not word in word2vec:
        create_vector(word, word2vec, word_vector_size, silent)
    if not word in vocab:
        next_index = len(vocab)
        vocab[word] = next_index
        ivocab[next_index] = word

    if to_return == "word2vec":
        return word2vec[word]
    elif to_return == "index":
        return vocab[word]
    elif to_return == "onehot":
        raise Exception("to_return = 'onehot' is not implemented yet")


def create_embedding(word2vec, ivocab, embed_size):
    embedding = np.zeros((len(ivocab), embed_size))
    for i in range(len(ivocab)):
        word = ivocab[i]
        embedding[i] = word2vec[word]
    return embedding


def process_input(data_raw, floatX, word2vec, vocab, ivocab, embed_size, split_sentences=False):
    questions = []
    inputs = []
    answers = []
    input_masks = []
    relevant_labels = []
    for x in data_raw:
        if split_sentences:
            inp = x["C"].lower().split(' . ')
            inp = [w for w in inp if len(w) > 0]
            inp = [i.split() for i in inp]
        else:
            inp = x["C"].lower().split(' ')
            inp = [w for w in inp if len(w) > 0]

        q = x["Q"].lower().split(' ')
        q = [w for w in q if len(w) > 0]

        if split_sentences:
            inp_vector = [[process_word(word = w,
                                        word2vec = word2vec,
                                        vocab = vocab,
                                        ivocab = ivocab,
                                        word_vector_size = embed_size,
                                        to_return = "index") for w in s] for s in inp]
        else:
            inp_vector = [process_word(word = w,
                                        word2vec = word2vec,
                                        vocab = vocab,
                                        ivocab = ivocab,
                                        word_vector_size = embed_size,
                                        to_return = "index") for w in inp]

        q_vector = [process_word(word = w,
                                    word2vec = word2vec,
                                    vocab = vocab,
                                    ivocab = ivocab,
                                    word_vector_size = embed_size,
                                    to_return = "index") for w in q]

        if split_sentences:
            inputs.append(inp_vector)
        else:
            inputs.append(np.vstack(inp_vector).astype(floatX))
        questions.append(np.vstack(q_vector).astype(floatX))
        answers.append(process_word(word = x["A"],
                                        word2vec = word2vec,
                                        vocab = vocab,
                                        ivocab = ivocab,
                                        word_vector_size = embed_size,
                                        to_return = "index"))
        # NOTE: here we assume the answer is one word!

        if not split_sentences:
            if input_mask_mode == 'word':
                input_masks.append(np.array([index for index, w in enumerate(inp)], dtype=np.int32))
            elif input_mask_mode == 'sentence':
                input_masks.append(np.array([index for index, w in enumerate(inp) if w == '.'], dtype=np.int32))
            else:
                raise Exception("invalid input_mask_mode")

        relevant_labels.append(x["S"])

    return inputs, questions, answers, input_masks, relevant_labels

def get_lens(inputs, split_sentences=False):
    lens = np.zeros((len(inputs)), dtype=int)
    for i, t in enumerate(inputs):
        lens[i] = t.shape[0]
    return lens

def get_sentence_lens(inputs):
    lens = np.zeros((len(inputs)), dtype=int)
    sen_lens = []
    max_sen_lens = []
    for i, t in enumerate(inputs):
        sentence_lens = np.zeros((len(t)), dtype=int)
        for j, s in enumerate(t):
            sentence_lens[j] = len(s)
        lens[i] = len(t)
        sen_lens.append(sentence_lens)
        max_sen_lens.append(np.max(sentence_lens))
    return lens, sen_lens, max(max_sen_lens)


def pad_inputs(inputs, lens, max_len, mode="", sen_lens=None, max_sen_len=None):
    if mode == "mask":
        padded = [np.pad(inp, (0, max_len - lens[i]), 'constant', constant_values=0) for i, inp in enumerate(inputs)]
        return np.vstack(padded)

    elif mode == "split_sentences":
        padded = np.zeros((len(inputs), max_len, max_sen_len))
        for i, inp in enumerate(inputs):
            padded_sentences = [np.pad(s, (0, max_sen_len - sen_lens[i][j]), 'constant', constant_values=0) for j, s in enumerate(inp)]
            # trim array according to max allowed inputs
            if len(padded_sentences) > max_len:
                padded_sentences = padded_sentences[(len(padded_sentences)-max_len):]
                lens[i] = max_len
            padded_sentences = np.vstack(padded_sentences)
            padded_sentences = np.pad(padded_sentences, ((0, max_len - lens[i]),(0,0)), 'constant', constant_values=0)
            padded[i] = padded_sentences
        return padded

    padded = [np.pad(np.squeeze(inp, axis=1),
                     (0, max_len - lens[i]),
                     'constant',
                     constant_values=0)
              if lens[i] < max_len else np.squeeze(inp, axis=1)[:max_len]
              for i, inp in enumerate(inputs)]
    # old version: cannot handle case that original length longer than max_len
    # padded = [np.pad(np.squeeze(inp, axis=1), (0, max_len - lens[i]), 'constant', constant_values=0) for i, inp in enumerate(inputs)]
    return np.vstack(padded)


def load_babi(config, split_sentences=False):
    vocab = {}
    ivocab = {}

    babi_train_raw, babi_test_raw = get_babi_raw(config.babi_id, config.babi_test_id)

    if config.word2vec_init:
        assert config.embed_size == 100
        word2vec = load_glove(config.embed_size)
    else:
        word2vec = {}

    # set word at index zero to be end of sentence token so padding with zeros is consistent
    process_word(word = "<eos>",
                word2vec = word2vec,
                vocab = vocab,
                ivocab = ivocab,
                word_vector_size = config.embed_size,
                to_return = "index")

    print('==> get train inputs')
    train_data = process_input(babi_train_raw, config.floatX, word2vec, vocab, ivocab, config.embed_size, split_sentences)
    print('==> get test inputs')
    test_data = process_input(babi_test_raw, config.floatX, word2vec, vocab, ivocab, config.embed_size, split_sentences)

    if config.word2vec_init:
        assert config.embed_size == 100
        word_embedding = create_embedding(word2vec, ivocab, config.embed_size)
    else:
        word_embedding = np.random.uniform(-config.embedding_init, config.embedding_init, (len(ivocab), config.embed_size))

    inputs, questions, answers, input_masks, rel_labels = train_data if config.train_mode else test_data

    if split_sentences:
        input_lens, sen_lens, max_sen_len = get_sentence_lens(inputs)
        max_mask_len = max_sen_len
    else:
        input_lens = get_lens(inputs)
        mask_lens = get_lens(input_masks)
        max_mask_len = np.max(mask_lens)

    q_lens = get_lens(questions)

    max_q_len = np.max(q_lens)
    max_input_len = min(np.max(input_lens), config.max_allowed_inputs)

    #pad out arrays to max
    if split_sentences:
        inputs = pad_inputs(inputs, input_lens, max_input_len, "split_sentences", sen_lens, max_sen_len)
        input_masks = np.zeros(len(inputs))
    else:
        inputs = pad_inputs(inputs, input_lens, max_input_len)
        input_masks = pad_inputs(input_masks, mask_lens, max_mask_len, "mask")

    questions = pad_inputs(questions, q_lens, max_q_len)

    answers = np.stack(answers)

    rel_labels = np.zeros((len(rel_labels), len(rel_labels[0])))

    for i, tt in enumerate(rel_labels):
        rel_labels[i] = np.array(tt, dtype=int)

    if config.train_mode:
        train = (questions[:config.num_train], inputs[:config.num_train],
                q_lens[:config.num_train], input_lens[:config.num_train],
                input_masks[:config.num_train], answers[:config.num_train],
                rel_labels[:config.num_train])

        valid = (questions[config.num_train:], inputs[config.num_train:],
                q_lens[config.num_train:], input_lens[config.num_train:],
                input_masks[config.num_train:], answers[config.num_train:],
                rel_labels[config.num_train:])
        return train, valid, word_embedding, max_q_len, max_input_len, max_mask_len, rel_labels.shape[1], ivocab

    else:
        test = questions, inputs, q_lens, input_lens, input_masks, answers, rel_labels
        return test, word_embedding, max_q_len, max_input_len, max_mask_len, rel_labels.shape[1], ivocab

In [91]:
class Config(object):
    """模型与数据方面的超参数。"""

    batch_size = 100
    embed_size = 81
    hidden_size = 83

    max_epochs = 256
    early_stopping = 20

    dropout = 0.9
    lr = 0.001
    l2 = 0.001

    cap_grads = False
    max_grad_val = 10
    noisy_grads = False

    word2vec_init = False
    embedding_init = np.sqrt(3)

    drop_grus = False

    anneal_threshold = 1000
    anneal_by = 1.5

    num_hops = 3
    num_attention_features = 4

    max_allowed_inputs = 130
    num_train = 900

    floatX = np.float32

    babi_id = "1"
    babi_test_id = ""

    train_mode = True


## demo: bAbI dataset for task 2

In [141]:
config = Config()
results = load_babi(config, split_sentences=True)
train, valid, word_embedding, max_q_len, max_input_len, max_mask_len, rel_labels_shape_1, vocab = results

==> Loading test from /home/dong/Dropbox/Projects/NLP/memoryNet/bAbIdata/tasks_1-20_v1-2/en/qa1_single-supporting-fact_train.txt
==> Loading test from /home/dong/Dropbox/Projects/NLP/memoryNet/bAbIdata/tasks_1-20_v1-2/en/qa1_single-supporting-fact_test.txt
==> get train inputs
==> get test inputs


```python
if config.train_mode:
    train = (questions[:config.num_train],
             inputs[:config.num_train],
             q_lens[:config.num_train],
             input_lens[:config.num_train],
             input_masks[:config.num_train],
             answers[:config.num_train],
             rel_labels[:config.num_train])

    valid = (questions[config.num_train:],
             inputs[config.num_train:],
             q_lens[config.num_train:],
             input_lens[config.num_train:],
             input_masks[config.num_train:],
             answers[config.num_train:],
             rel_labels[config.num_train:])

    return train, valid, word_embedding, max_q_len, max_input_len, max_mask_len, rel_labels.shape[1], len(vocab)

else:
    test = questions, inputs, q_lens, input_lens, input_masks, answers, rel_labels
    return test, word_embedding, max_q_len, max_input_len, max_mask_len, rel_labels.shape[1], len(vocab)

```

In [143]:
print('shape of train-->question = %s' % repr(train[0].shape))
print('print 3 examples of question, in digits:')
print(repr(train[0][:3,:]))
print('print 3 examples of questions, in letters:')
for i in range(3):
    print('\t%s' % ' '.join([vocab[int(x)] for x in train[0][i,:]]))

print('\n')
print('shape of train-->q_lens = %s' % repr(train[2].shape))
print('print 3 examples of questino lengths:')
print(repr(train[2][:3]))

shape of train-->question = (900, 3)
print 3 examples of question, in digits:
array([[  9.,  10.,   1.],
       [  9.,  10.,  11.],
       [  9.,  10.,  11.]], dtype=float32)
print 3 examples of questions, in letters:
	where is mary
	where is daniel
	where is daniel


shape of train-->q_lens = (900,)
print 3 examples of questino lengths:
array([3, 3, 3])


In [146]:
print('\nshape of train-->inputs = %s:' % repr(train[1].shape))
print('print 3 examples of inputs, in digits:')
print(repr(train[1][:3,:]))
print('print 3 examples of inputs, in letters:')
for i in range(3):
    print('example %d' % i)
    for j in range(max_mask_len):
        print('\t%s' % ' '.join([vocab[int(x)] for x in train[1][i,j,:]]))
    
print('\n')
print('shape of train-->input_lens = %s' % repr(train[3].shape))
print('print 3 examples of inputs lengths:')
print(repr(train[3][:3]))

print('\n')
print('shape of train-->input_masks_lens = %s' % repr(train[4].shape))
print('print 3 examples of inputs_masks lengths:')
print(repr(train[4][:3]))


shape of train-->inputs = (900, 10, 6):
print 3 examples of inputs, in digits:
array([[[  1.,   2.,   3.,   4.,   5.,   0.],
        [  6.,   7.,   3.,   4.,   8.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.]],

       [[  1.,   2.,   3.,   4.,   5.,   0.],
        [  6.,   7.,   3.,   4.,   8.,   0.],
        [ 11.,   7.,  12.,   3.,   4.,   8.],
        [ 13.,   2.,   3.,   4.,  14.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.]

In [126]:
print('\nshape of train-->answers = %s:' % repr(train[5].shape))
print('print 3 examples of answers, in digits:')
print(repr(train[5][:3]))
print('print 3 examples of answers, in letters:')
print('%s' % ', '.join([vocab[int(x)] for x in train[5][:3]]))



shape of train-->answers = (900,):
print 3 examples of answers, in digits:
array([5, 8, 8])
print 3 examples of answers, in letters:
bathroom, hallway, hallway


In [145]:
print('\nmax_q_len=%d\nmax_input_len=%d\nmax_sen_len=%d' % (max_q_len, max_input_len, max_mask_len))


max_q_len=3
max_input_len=10
max_sen_len=6


```
 /\
/  \
 ||
```
在`split_sentence=True`的设定下：
#### `max_sen_len`是单个输入句子的长度上限
#### `max_input_len`是输入事实中句子数量的上限
即，一个样本的input中包含的句子的数量的上限
#### `max_q_len`是问题句子的长度上限

## demo: bAbI dataset for task 19

In [131]:
config = Config()
config.babi_id = "19"
results = load_babi(config, split_sentences=True)
train, valid, word_embedding, max_q_len, max_input_len, max_mask_len, rel_labels_shape_1, vocab = results

==> Loading test from /home/dong/Dropbox/Projects/NLP/memoryNet/bAbIdata/tasks_1-20_v1-2/en/qa19_path-finding_train.txt
==> Loading test from /home/dong/Dropbox/Projects/NLP/memoryNet/bAbIdata/tasks_1-20_v1-2/en/qa19_path-finding_test.txt
==> get train inputs
==> get test inputs


In [132]:
print('shape of train-->question = %s' % repr(train[0].shape))
print('print 3 examples of question, in digits:')
print(repr(train[0][:3,:]))
print('print 5 examples of questions, in letters:')
for i in range(3):
    print('\t%s' % ' '.join([vocab[int(x)] for x in train[0][i,:]]))

print('\n')
print('shape of train-->q_lens = %s' % repr(train[2].shape))
print('print 3 examples of questino lengths:')
print(repr(train[2][:3]))

shape of train-->question = (900, 10)
print 3 examples of question, in digits:
array([[ 13.,  14.,  15.,  16.,  17.,   1.,   7.,  18.,   1.,   9.],
       [ 13.,  14.,  15.,  16.,  17.,   1.,   7.,  18.,   1.,   6.],
       [ 13.,  14.,  15.,  16.,  17.,   1.,   9.,  18.,   1.,  11.]], dtype=float32)
print 5 examples of questions, in letters:
	how do you go from the kitchen to the garden
	how do you go from the kitchen to the hallway
	how do you go from the garden to the bedroom


shape of train-->q_lens = (900,)
print 3 examples of questino lengths:
array([10, 10, 10])


In [137]:
print('\nshape of train-->inputs = %s:' % repr(train[1].shape))
print('print 3 examples of inputs, in digits:')
print(repr(train[1][:3,:]))
print('print 3 examples of inputs, in letters:')
for i in range(3):
    print('example %d' % i)
    for j in range(len(train[1][0])):
        print('\t%s' % ' '.join([vocab[int(x)] for x in train[1][i,j,:]]))
    
print('\n')
print('shape of train-->input_lens = %s' % repr(train[3].shape))
print('print 3 examples of inputs lengths:')
print(repr(train[3][:3]))


shape of train-->inputs = (900, 5, 7):
print 3 examples of inputs, in digits:
array([[[  1.,   2.,   3.,   4.,   5.,   1.,   6.],
        [  1.,   7.,   3.,   8.,   5.,   1.,   2.],
        [  1.,   9.,   3.,  10.,   5.,   1.,  11.],
        [  1.,   2.,   3.,  10.,   5.,   1.,   9.],
        [  1.,  12.,   3.,   8.,   5.,   1.,   9.]],

       [[  1.,  11.,   3.,  10.,   5.,   1.,   6.],
        [  1.,   2.,   3.,   4.,   5.,   1.,   9.],
        [  1.,   9.,   3.,   8.,   5.,   1.,   7.],
        [  1.,   7.,   3.,   8.,   5.,   1.,  12.],
        [  1.,   6.,   3.,  10.,   5.,   1.,   9.]],

       [[  1.,  11.,   3.,  21.,   5.,   1.,   6.],
        [  1.,  12.,   3.,   4.,   5.,   1.,   2.],
        [  1.,   7.,   3.,  10.,   5.,   1.,   9.],
        [  1.,   9.,   3.,  21.,   5.,   1.,   2.],
        [  1.,   2.,   3.,  21.,   5.,   1.,  11.]]])
print 3 examples of inputs, in letters:
example 0
	the office is east of the hallway
	the kitchen is north of the office
	the garden is

In [138]:
print('\nshape of train-->answers = %s:' % repr(train[5].shape))
print('print 3 examples of answers, in digits:')
print(repr(train[5][:3]))
print('print 3 examples of answers, in letters:')
print('%s' % '\t'.join([vocab[int(x)] for x in train[5][:3]]))


shape of train-->answers = (900,):
print 3 examples of answers, in digits:
array([19, 20, 22])
print 3 examples of answers, in letters:
s,e	n,w	n,n


In [139]:
print('\nmax_q_len=%d\nmax_input_len=%d' % (max_q_len, max_input_len))


max_q_len=10
max_input_len=5
